In [10]:
import json
import pandas as pd
import dill as pickle

class DataUtil2:
    """"""
    @classmethod
    def load(cls, file, format, date_cols= None):
        """"""
        data = ''
        if(format== 'csv'):
            data = pd.read_csv(file, parse_dates= date_cols)
        elif(format== 'json'):
            with open(file, 'r') as i_file:
                data = json.load(file)
            i_file.close()
        elif(format== 'pkl'):
            with open(file, 'rb') as i_file:
                data = pickle.load(i_file)
            i_file.close()
        elif(format == 'hdf'):
            data = pd.read_hdf(path_or_buf= file, key='undefined')

        return  data

    @classmethod
    def save(cls, data, file, format, precision= 8):
        """"""
        if(format == 'csv'):
            data.to_csv(file, float_format= '%%.%df' % precision, index= False)
        elif(format == 'json'):
            with open(file, 'w') as o_file:
                json.dump(data, o_file, ensure_ascii= True, indent= 4)
            o_file.close()
        elif(format == 'pkl'):
            with open(file, 'wb') as o_file:
                pickle.dump(data, o_file, -1)
            o_file.close()
        elif(format== 'hdf'):
            data.to_hdf(path_or_buf= file, key='undefined', mode='w', complib='blosc')

        return

In [24]:
import pandas as pd # for aristhmetic computing
import numpy as np 
import math
import datetime # for timing
import os,sys,gc # for debuging
import re, string
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tag import pos_tag

import time
import re

# replacements
replacement_patterns = [
     (r'won\'t', 'will not'),
     (r'can\'t', 'cannot'),
     (r'i\'m', 'i am'),
     (r'ain\'t', 'is not'),
     (r'(\w+)\'ll', '\g<1> will'),
     (r'(\w+)n\'t', '\g<1> not'),
     (r'(\w+)\'ve', '\g<1> have'),
     (r'(\w+)\'s', '\g<1> is'),
     (r'(\w+)\'re', '\g<1> are'),
     (r'(\w+)\'d', '\g<1> would')
]
class RegexpReplacer(object):
    def __init__(self, patterns=replacement_patterns):
        self.patterns = [(re.compile(regex), repl) for (regex, repl) in patterns]
    def replace(self, text):
        s = text
        for (pattern, repl) in self.patterns:
            s = re.sub(pattern, repl, s)
        return s
# special characters
special_character_removal= re.compile(r'[^a-z\d]', re.IGNORECASE)
# numeric characters
replace_numbers= re.compile(r'\d+', re.IGNORECASE)
# stop words
stopwords_set = set(stopwords.words('english'))
# lemmatization
def lemmatize_all(sentence):
    wnl = WordNetLemmatizer()
    for word, tag in pos_tag(sentence):
        if tag.startswith("NN"):
            yield wnl.lemmatize(word, pos='n')
        elif tag.startswith('VB'):
            yield wnl.lemmatize(word, pos='v')
        elif tag.startswith('JJ'):
            yield wnl.lemmatize(word, pos='a')
# process rows
def ApplyBOW(ColValues):
    ''''''
    n = len(ColValues)
    result = ['' for i in range(n)]
    i = 0
    print('total length %s.' % n)
    start = time.time()
    for i in range(n):
        end = time.time()
        if(i % 5000 == 0):
            print('%s done, time elapsed %.2fs.' % (i, (end - start)))
        row = str(ColValues[i])
        # replacement
        replacer = RegexpReplacer()
        row = replacer.replace(row)
        #Remove Special Characters
        row = special_character_removal.sub(' ', row)
        #Replace Numbers
        row = replace_numbers.sub('n', row)
        # remove punctations
        tokenizer = RegexpTokenizer(r"[\w']+")
        if(row == np.nan):
            word_vec = []
        try:
            word_vec = tokenizer.tokenize(row)
        except Exception as e:
            print(type(row))
            sys.exit(1)
        # to lower
        word_vec = [word.lower() for word in word_vec]

        # remove stopwords
        word_vec = [word for word in word_vec if(word not in stopwords_set)]
        # lemmatization
        word_vec = lemmatize_all(word_vec)
        # length > 1
        word_vec = [w for w in word_vec if(len(w) > 1)]
        # join
        text = ' '.join(word_vec)
        result[i] = text
        i += 1
    return result
# data source   
DataBase = '../data'
DataSet = {
    'train': pd.read_csv('%s/raw/train.csv' % DataBase),
    'test': pd.read_csv('%s/raw/test.csv' % DataBase),
    #'sub': pd.read_csv('%s/raw/sample_submission.csv' % DataBase)
}
# save
OutputDir = '%s/clean' % DataBase
if(os.path.exists(OutputDir) == False):
    os.makedirs(OutputDir)
COMMENT = 'comment_text'
label_cols = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
for col in label_cols:
    DataSet['test'][col] = 0
# entry
# #TODO
# np.random.seed(2017)
# DataSet['train'] = DataSet['train'].sample(frac= 0.1)
# DataSet['test'] = DataSet['test'].sample(frac= 0.1)
for mod in ['train', 'test']:
    s = time.time()
    # fill null comment text
    DataSet[mod][COMMENT].fillna("", inplace= True)
    # process rows
    DataSet[mod][COMMENT] = ApplyBOW(DataSet[mod][COMMENT].values)
    # log1p length
    #DataSet[mod]['total_log1p_len'] = np.log1p(DataSet[mod][COMMENT].map(lambda x: len(x.split(' '))))
    # none label
    DataSet[mod]['none'] = 1 - DataSet[mod][label_cols].max(axis=1)
    # save
    DataSet[mod][:200].to_csv('%s/%s.csv' % (OutputDir, mod), index= False)
    DataUtil2.save(DataSet[mod], '%s/%s.%s' % (OutputDir, mod, 'hdf'), 'hdf')
    e = time.time()
    print('time %.2fs' % ((e - s)))

total length 95851.
0 done, time elapsed 0.00s.
5000 done, time elapsed 7.88s.
10000 done, time elapsed 15.62s.
15000 done, time elapsed 23.45s.
20000 done, time elapsed 31.56s.
25000 done, time elapsed 38.80s.
30000 done, time elapsed 46.34s.
35000 done, time elapsed 54.20s.
40000 done, time elapsed 62.00s.
45000 done, time elapsed 69.43s.
50000 done, time elapsed 77.00s.
55000 done, time elapsed 85.24s.
60000 done, time elapsed 92.66s.
65000 done, time elapsed 100.07s.
70000 done, time elapsed 107.34s.
75000 done, time elapsed 115.29s.
80000 done, time elapsed 123.35s.
85000 done, time elapsed 131.17s.
90000 done, time elapsed 139.19s.
95000 done, time elapsed 147.67s.
time 149.18s
total length 226998.
0 done, time elapsed 0.00s.
5000 done, time elapsed 9.38s.
10000 done, time elapsed 30.80s.
15000 done, time elapsed 40.95s.
20000 done, time elapsed 49.40s.
25000 done, time elapsed 58.33s.
30000 done, time elapsed 67.23s.
35000 done, time elapsed 76.75s.
40000 done, time elapsed 85.3